In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
user_input = "나 곧 졸업해"

prompt = f"""
당신은 꽃집 사장님입니다. 사용자의 감정이나 상황에 따라 어울리는 꽃 색상 2개를 제시해주세요. 색상만 쉼표로 구분해서 단답형으로 대답해주세요.

예시:
입력: 우울한 나에게 위로가 될 꽃을 사고 싶어요
출력: 연보라, 핑크

입력: 축하해주고 싶은 기분이에요
출력: 노랑, 빨강

입력: {user_input}
출력:
"""

In [10]:
import torch

inputs = tokenizer(prompt, return_tensors="pt").to("mps")  
model.to("mps")

with torch.no_grad():
    generated = model.generate(
        inputs.input_ids,
        max_new_tokens=20,
        do_sample=True, 
        temperature=0.7,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
    )

output_text = tokenizer.decode(generated[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/opt/anaconda3/envs/torch/lib/python3.9/site-packages/transformers/pytorch_utils.py:333: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


In [12]:
# 출력 문자열에서 마지막 '출력:' 이후만 추출
if "출력:" in output_text:
    result = output_text.split("출력:")[-1].strip().split("\n")[0]
else:
    result = output_text.strip()

print("추천 색상:", result)


추천 색상: '''


In [15]:
import json

with open("coloremotion.json", encoding="utf-8") as f:
    db = json.load(f)

docs = []
for color, meta in db.items():
    sent = (
        f"{color} 색은 {','.join(meta['emotion'])} 감정을 "
        f"유발하며 {', '.join(meta['associations'])} 느낌이 있다."
    )
    docs.append({"id": color, "text": sent})


In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb
emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5") 
chroma = chromadb.Client()
col = chroma.create_collection("color_psych")
for d in docs:
    emb = emb_model.encode(d["text"]).tolist()
    col.add(ids=d["id"], embeddings=emb, documents=d["text"])


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [27]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T")

In [37]:
import os
import json
import re
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
import chromadb
from sentence_transformers import SentenceTransformer

# ── 0. 환경 변수 설정 (Mac MPS 메모리 한도 해제) ────────────
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

# ── 1. 색상‐감정 JSON 불러오기 ───────────────────────────────
with open("coloremotion.json", encoding="utf-8") as f:
    db = json.load(f)

# ── 2. (생략) 이미 만들어둔 chroma 컬렉션 재사용 ────────────
chroma = chromadb.Client()
col    = chroma.get_collection("color_psych")

# ── 3. TinyLlama 모델 로드 ───────────────────────────────────
model_id = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
device   = "mps" if torch.backends.mps.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model     = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16
).to(device)
model.eval()

# ── 4. 직접 언급된 색상 추출 유틸 ────────────────────────────
def extract_explicit_colors(text: str):
    return [c for c in db.keys() if c in text.lower()]

# ── 5. RAG + 로컬 generate 함수 ─────────────────────────────────
emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5")

def recommend_colors_local(user_text: str, top_k: int = 3) -> str:
    # (a) 직접 언급된 색상이 있으면 우선 반환
    explicit = extract_explicit_colors(user_text)
    if explicit:
        return ", ".join(explicit[:2])

    # (b) 벡터 검색으로 context 구축
    q_emb   = emb_model.encode(user_text).tolist()
    hits    = col.query(query_embeddings=[q_emb], n_results=top_k)
    context = " | ".join(hits["documents"][0])

    # (c) 프롬프트 작성
    prompt = (
        f"너는 색상 심리학 기반 꽃다발 색상 추천 전문가야.\n"
        f"다음 정보를 참고해 사용자에게 어울리는 꽃 색상 두 개만 쉼표로 구분해 추천해줘.\n\n"
        f"색상 정보: {context}\n"
        f"사용자 요청: \"{user_text}\"\n"
        f"출력:"
    )

    # (d) 토크나이즈 및 generate
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=32,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id
        )

    # (e) 결과 파싱
    full = tokenizer.decode(out[0], skip_special_tokens=True)
    answer = full.split("출력:")[-1].strip().splitlines()[0]
    return answer

# ── 6. 테스트 ───────────────────────────────────────────────
if __name__ == "__main__":
    tests = [
        "친구가 요즘 우울해 보여서 위로 꽃을 주고 싶어",
        "노란 꽃다발이 끌려요",
        "생일 축하 꽃을 사고 싶어요",
        "안정감을 주는 평온한 느낌의 꽃이 필요해요"
    ]
    for t in tests:
        print(f"입력: {t}\n추천: {recommend_colors_local(t)}\n")


RuntimeError: MPS backend out of memory (MPS allocated: 18.10 GB, other allocations: 2.41 MB, max allowed: 18.13 GB). Tried to allocate 125.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).